# Занятие 6.1: Введение в обработку текста

Максим Ионов, [max.ionov@gmail.com](mailto:max.ionov@gmail.com)

## Анализ сентиментов в твиттере

In [58]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import sklearn.pipeline
import sklearn.naive_bayes
from nltk.tokenize import RegexpTokenizer
import sklearn.metrics
import nltk

In [52]:
colnames = ['id', 'tdate', 'tmane', 'ttext', 'ttype', 'trep','tftw', 'tfav', 'tstcount', 'tfol', 'tfrien', 'listcount']
pos_data = pd.read_csv('positive.csv', names=colnames, sep=';')
neg_data = pd.read_csv('negative.csv', names=colnames, sep=';')

In [13]:
pos_data.head()

,id,tdate,tmane,ttext,ttype,trep,tftw,tfav,tstcount,tfol,tfrien,listcount
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1


In [53]:
data = pd.concat([pos_data, neg_data] ).reset_index()
train_data, test_data = train_test_split(data, test_size=0.2)
train_data, val_data = train_test_split(train_data, test_size=0.2)

In [63]:
textcolumn= 'ttext'
sentcolumn = 'ttype'
def getconfmatrix(text_clf, train_data, val_data):
    text_clf.fit(train_data[textcolumn], train_data[sentcolumn])
    predicted = text_clf.predict(val_data[textcolumn])
    print(sklearn.metrics.classification_report(val_data[sentcolumn], predicted))
    return sklearn.metrics.confusion_matrix(val_data[sentcolumn], predicted)

### tokenizer - split ' ' + countVectorizer

In [46]:
text_clf = sklearn.pipeline.Pipeline([('vect', 
                                       sklearn.feature_extraction.text.CountVectorizer(min_df=5, max_df=0.75, 
                                                 tokenizer=lambda s: s.split(' '))),
                     ('clf', sklearn.naive_bayes.MultinomialNB())])
getconfmatrix(text_clf, train_data, val_data)

             precision    recall  f1-score   support

         -1       0.83      0.84      0.83     17900
          1       0.84      0.83      0.84     18394

avg / total       0.84      0.84      0.84     36294



array([[15028,  2872],
       [ 3101, 15293]])

### countvectorizer

In [47]:
text_clf = sklearn.pipeline.Pipeline([('vect', 
                                       sklearn.feature_extraction.text.TfidfVectorizer(min_df=5, max_df=0.75)),
                     ('clf', sklearn.naive_bayes.MultinomialNB())])
getconfmatrix(text_clf, train_data, val_data)

             precision    recall  f1-score   support

         -1       0.74      0.75      0.74     17900
          1       0.75      0.74      0.75     18394

avg / total       0.75      0.75      0.75     36294



array([[13432,  4468],
       [ 4739, 13655]])

### CountVectorizer + ngram(1,2)

In [48]:
text_clf = sklearn.pipeline.Pipeline([('vect', 
                                       sklearn.feature_extraction.text.TfidfVectorizer(min_df=5, max_df=0.75)),
                     ('clf', sklearn.naive_bayes.MultinomialNB())])
getconfmatrix(text_clf, train_data, val_data)

             precision    recall  f1-score   support

         -1       0.74      0.75      0.74     17900
          1       0.75      0.74      0.75     18394

avg / total       0.75      0.75      0.75     36294



array([[13432,  4468],
       [ 4739, 13655]])

### tokenizer Regexp W (дает прирост в 0.2 относительно w+)

In [64]:
def tryDifferenTokenizers(getWords, train_data, val_data):
    text_clf = sklearn.pipeline.Pipeline([('vect', 
                                       sklearn.feature_extraction.text.CountVectorizer(min_df=5, max_df=0.75,
                                      tokenizer=lambda s: getWords(s))),
                     ('clf', sklearn.naive_bayes.MultinomialNB())])
    return getconfmatrix(text_clf, train_data, val_data)


tokenizer = RegexpTokenizer(r'\W')
def getWordsNltkRegExp(s):
    return [word.lower() for word in tokenizer.tokenize(s)]
tryDifferenTokenizers(getWordsNltkRegExp, train_data, val_data)

             precision    recall  f1-score   support

         -1       0.98      0.97      0.97     17857
          1       0.97      0.98      0.97     18437

avg / total       0.97      0.97      0.97     36294



array([[17262,   595],
       [  368, 18069]])

In [59]:
tweetTokenizer = nltk.tokenize.casual.TweetTokenizer(preserve_case=False)
def getWordsTweetTokenizer(s):
     return tweetTokenizer.tokenize(s)
tryDifferenTokenizers(getWordsTweetTokenizer, train_data, val_data)

             precision    recall  f1-score   support

         -1       1.00      0.99      1.00     17857
          1       0.99      1.00      1.00     18437

avg / total       1.00      1.00      1.00     36294



array([[17706,   151],
       [   26, 18411]])

### Conf_matrix - для tweetTokenizer на всех данных

In [66]:
tryDifferenTokenizers(getWordsTweetTokenizer, train_data, data)

             precision    recall  f1-score   support

         -1       1.00      0.99      1.00    111923
          1       0.99      1.00      1.00    114911

avg / total       1.00      1.00      1.00    226834



array([[111090,    833],
       [   138, 114773]])

### Conf_matrix для nltkTokenizer на всех данных

In [67]:
tryDifferenTokenizers(getWordsNltkRegExp, train_data, data)

             precision    recall  f1-score   support

         -1       0.98      0.97      0.97    111923
          1       0.97      0.98      0.97    114911

avg / total       0.97      0.97      0.97    226834



array([[108157,   3766],
       [  2406, 112505]])

### Посмотрим, на каких предложениях ошибся классификатор с tweetTokenizer

In [ ]:
text_clf = sklearn.pipeline.Pipeline([('vect', 
                                       sklearn.feature_extraction.text.CountVectorizer(min_df=5, max_df=0.75,
                                      tokenizer=lambda s: getWordsTweetTokenizer(s))),
                     ('clf', sklearn.naive_bayes.MultinomialNB())])
text_clf.fit(train_data[textcolumn], train_data[sentcolumn])
predicted = text_clf.predict(val_data[textcolumn])

In [51]:
pd.set_option('display.max_colwidth', 1000)
val_data[val_data[sentcolumn]!= predicted][[textcolumn, sentcolumn]]

,ttext,ttype
179984,RT @pro_o_pro: Константин дразнит Олю http://t.co/Ewl4oj8tby #новости #Селигер2013,-1
133855,RT @Lehailigde: Снег крадут=*( http://t.co/4cNBQ21qbY,-1
158686,Ахаха! Дибилы бесите со своими концами...света! }:-/ каждый второй статус ёб...!,-1
212858,RT @Hi_natasha_: #МолСовет БВ на #СеребрянаяНить @vp1101156 @id_karo_o @aleksey_perec http://t.co/d15rAwxBWt,-1
132787,RT @antje_O_o: работница месяца @tbushmel http://t.co/PEz2HRCLVI,-1
212448,RT @orko_ori: #драйвер для usb texet скачать http://t.co/WuBV0kOJTy,-1
191900,RT @jared_na_marse: Опаньки\n о_О http://t.co/1USmPFiFrt,-1
185252,RT @leaving_to_Omsk: завтра будет голди хастл пати ин да канищево гетто,-1
160917,"@blchk434 наверное,но мне кажется,когда мы были на прошлых Диксах,ты про нее спросила о_О ну да ладно,не важно ХД",-1
198835,@zO_Om это по какой шкале? По нашей или по ихней?,-1


### re split w+

In [33]:
import re
def getWords(s):
    
    return [word.lower() for word in re.split(r"\w+", s)]
    
text_clf = sklearn.pipeline.Pipeline([('vect', 
                                       sklearn.feature_extraction.text.CountVectorizer(min_df=5, max_df=0.75,
                                      tokenizer=lambda s: getWords(s))),
                     ('clf', sklearn.naive_bayes.MultinomialNB())])
getconfmatrix(text_clf, train_data, val_data)

             precision    recall  f1-score   support

         -1       0.97      0.95      0.96     17858
          1       0.95      0.97      0.96     18436

avg / total       0.96      0.96      0.96     36294



array([[16876,   982],
       [  475, 17961]])